Next Steps

    Tune the C parameter.
    Try different values of k (k-mer size).
    Optimize performance (e.g., using sparse representations for large sequences).

In [1]:
import numpy as np
import pandas as pd
import cvxopt
from collections import Counter

def generate_kmers(sequence, k):
    """Extract all k-mers from a sequence."""
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

def build_kmer_dict(sequences, k):
    """Create a dictionary mapping each unique k-mer to an index."""
    kmer_set = set()
    for seq in sequences:
        kmer_set.update(generate_kmers(seq, k))
    return {kmer: idx for idx, kmer in enumerate(sorted(kmer_set))}

def compute_kmer_feature_matrix(sequences, k, kmer_dict):
    """Convert sequences into k-mer frequency vectors."""
    num_samples = len(sequences)
    num_kmers = len(kmer_dict)
    feature_matrix = np.zeros((num_samples, num_kmers))

    for i, seq in enumerate(sequences):
        kmer_counts = Counter(generate_kmers(seq, k))
        for kmer, count in kmer_counts.items():
            if kmer in kmer_dict:
                feature_matrix[i, kmer_dict[kmer]] = count
    return feature_matrix

def spectrum_kernel_matrix(sequences1, sequences2, k):
    """Compute the Spectrum Kernel matrix between two sets of sequences."""
    all_sequences = np.concatenate([sequences1, sequences2])
    kmer_dict = build_kmer_dict(all_sequences, k)

    # Convert sequences to feature matrices
    X1 = compute_kmer_feature_matrix(sequences1, k, kmer_dict)
    X2 = compute_kmer_feature_matrix(sequences2, k, kmer_dict)

    return X1 @ X2.T  # Compute the dot product

def train_svm(K, y, C=1.0):
    """Train an SVM using the precomputed kernel matrix K."""
    n = len(y)
    y = y.astype(float).reshape(-1, 1)  # Ensure y is a column vector

    # Construct the quadratic programming matrices
    P = cvxopt.matrix(np.outer(y, y) * K)  # P_ij = y_i * y_j * K_ij
    q = cvxopt.matrix(-np.ones(n))        # q_i = -1
    G = cvxopt.matrix(np.vstack((-np.eye(n), np.eye(n))))  # Constraints 0 <= α <= C
    h = cvxopt.matrix(np.hstack((np.zeros(n), C * np.ones(n))))
    A = cvxopt.matrix(y.T)  # Equality constraint sum(α_i * y_i) = 0
    b = cvxopt.matrix(0.0)

    # Solve the quadratic program
    solution = cvxopt.solvers.qp(P, q, G, h, A, b)
    alphas = np.ravel(solution['x'])

    # Support vectors: α > 0
    sv_indices = alphas > 1e-5
    support_vectors = np.where(sv_indices)[0]
    alphas = alphas[sv_indices]
    support_y = y[sv_indices].flatten()

    # Compute the bias term (intercept)
    bias = np.mean(support_y - np.sum(alphas * support_y * K[support_vectors][:, support_vectors], axis=1))

    return alphas, support_vectors, bias

def predict_svm(K_test, alphas, support_vectors, support_y, bias):
    """Make predictions using the trained SVM."""
    return np.sign(np.sum(alphas * support_y * K_test[:, support_vectors], axis=1) + bias)


In [2]:
import numpy as np
import pandas as pd
import cvxopt
import optuna
from collections import Counter
cvxopt.solvers.options["show_progress"] = False


def load_data(seq_file: str, label_file: str) -> tuple:
    """
    加载多组CSV文件并合并数据
    
    参数:
        seq_files: 多个x.csv文件路径列表（如["x.csv", "x1.csv"]）
        label_files: 多个y.csv文件路径列表（如["y.csv", "y1.csv"]）
    
    返回:
        (sequences, labels): 合并后的序列列表和标签数组
    """
    # 合并所有数据
    df_seq = pd.read_csv(seq_file)
    df_label = pd.read_csv(label_file)
    
    merged = pd.merge(df_seq, df_label, on='Id', how='inner')
    if merged.empty:
        print(f"警告: {seq_file}和{label_file}中没有匹配的ID")

    # 提取数据
    sequences = merged['seq'].values
    labels = np.where(merged['Bound'] == 1, 1, -1)
    return sequences, labels

def manual_kfold_split(X, y, n_splits=3, seed=42):
    """Manually splits X and y into K folds for cross-validation."""
    np.random.seed(seed)
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    
    folds = np.array_split(indices, n_splits)
    return folds

def cross_val_score_manual(X, y, k, C, n_splits=3):
    """Performs cross-validation without using sklearn."""
    folds = manual_kfold_split(X, y, n_splits)
    accuracies = []

    for i in range(n_splits):
        # print("i = ", i)
        val_indices = folds[i]  # Current fold is validation set
        train_indices = np.hstack([folds[j] for j in range(n_splits) if j != i])  # Rest are training

        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]

        # Compute kernel matrices
        K_train = spectrum_kernel_matrix(X_train, X_train, k)
        K_val = spectrum_kernel_matrix(X_val, X_train, k)

        # Train and predict
        alphas, support_vectors, bias = train_svm(K_train, y_train, C)
        predictions = predict_svm(K_val, alphas, support_vectors, y_train[support_vectors], bias)

        # Compute accuracy
        accuracy = np.mean(predictions == y_val)
        accuracies.append(accuracy)

        # print("accuracies: ", accuracies)
    return np.mean(accuracies)

def objective(trial):
    """Objective function for Optuna to optimize k and C."""
    k = trial.suggest_int("k", 2, 5)  # k-mer length between 2 and 6
    C = trial.suggest_loguniform("C", 1e-3, 1e0)  # C in [0.01, 100]

    cross_val = cross_val_score_manual(X_train, Y_train, k, C)

    print(f"C = {C}, k = {k}: {cross_val}")
    return cross_val

def train_and_predict_spectrum_svm(X_train_path, Y_train_path, X_test_path, n_trials=20):
    """Hyperparameter optimization with Optuna, then train and predict."""
    global X_train, Y_train  # Needed for Optuna's objective function
    X_train, Y_train = load_data(X_train_path, Y_train_path)

    df_pred = pd.read_csv(X_test_path)
    X_test = df_pred["seq"].values
    # X_test = load_data(X_test_path)

    # Run Optuna optimization
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    # Best hyperparameters
    best_k = study.best_params["k"]
    best_C = study.best_params["C"]
    print(f"Best k: {best_k}, Best C: {best_C}")

    # Train final model with best hyperparameters
    K_train = spectrum_kernel_matrix(X_train, X_train, best_k)
    K_test = spectrum_kernel_matrix(X_test, X_train, best_k)
    alphas, support_vectors, bias = train_svm(K_train, Y_train, best_C)

    # Predict on test set
    predictions = predict_svm(K_test, alphas, support_vectors, Y_train[support_vectors], bias)

    # Convert {-1,1} predictions to {0,1}
    predictions = (predictions + 1) // 2

    df_pred["Bound"] = predictions

    return df_pred

In [ ]:

# === Run the function with hyperparameter tuning ===
prediction_0 = train_and_predict_spectrum_svm("./data/Xtr0.csv", "./data/Ytr0.csv", "./data/Xte0.csv", n_trials=20)
prediction_1 = train_and_predict_spectrum_svm("./data/Xtr1.csv", "./data/Ytr1.csv", "./data/Xte1.csv", n_trials=20)
prediction_2 = train_and_predict_spectrum_svm("./data/Xtr2.csv", "./data/Ytr2.csv", "./data/Xte2.csv", n_trials=20)

[I 2025-02-21 19:14:18,324] A new study created in memory with name: no-name-931261fe-99e4-4d94-abe0-95374905a67f
C:\Users\liuce\AppData\Local\Temp\ipykernel_13288\1816469161.py:73: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 1e0)  # C in [0.01, 100]
[I 2025-02-21 19:14:26,248] Trial 0 finished with value: 0.5699897798848323 and parameters: {'k': 2, 'C': 0.4860433805245759}. Best is trial 0 with value: 0.5699897798848323.


C = 0.4860433805245759, k = 2: 0.5699897798848323


[I 2025-02-21 19:14:33,174] Trial 1 finished with value: 0.5820062941502222 and parameters: {'k': 3, 'C': 0.7765205675026385}. Best is trial 1 with value: 0.5820062941502222.


C = 0.7765205675026385, k = 3: 0.5820062941502222


[I 2025-02-21 19:14:39,748] Trial 2 finished with value: 0.6000138069103587 and parameters: {'k': 4, 'C': 0.037963787831821846}. Best is trial 2 with value: 0.6000138069103587.


...

C = 0.031107143284761617, k = 4: 0.6125023074048562


[I 2025-02-21 19:21:57,746] Trial 18 finished with value: 0.6295118206662434 and parameters: {'k': 5, 'C': 0.007925224469777689}. Best is trial 18 with value: 0.6295118206662434.


C = 0.007925224469777689, k = 5: 0.6295118206662434


[I 2025-02-21 19:22:05,461] Trial 19 finished with value: 0.6255108181644914 and parameters: {'k': 5, 'C': 0.00940072197718209}. Best is trial 18 with value: 0.6295118206662434.


C = 0.00940072197718209, k = 5: 0.6255108181644914
Best k: 5, Best C: 0.007925224469777689


In [7]:
def save_to_csv(df_0, df_1, df_2, suffixe = ""):
    res = pd.concat([df_0, df_1, df_2], ignore_index = True)
    res["Bound"] = res["Bound"].astype("int")
    res[["Id", "Bound"]].to_csv(f"./results/predictions_{suffixe}.csv", index = False)
    return
    # df_pred.to_csv("predictions.csv"

In [20]:
save_to_csv(prediction_0, prediction_1, prediction_2)

# Spectrum kernel with logistic regression

In [8]:
def sigmoid(z):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-z))

def train_logistic_regression(K_train, y_train, C, lr=0.01, epochs=1000):
    """
    Train logistic regression using batch gradient descent.
    
    Parameters:
    - K_train: (n_samples, n_samples) kernel matrix
    - y_train: (n_samples,) labels {-1,1}
    - C: regularization parameter
    - lr: learning rate
    - epochs: number of gradient updates
    
    Returns:
    - w: optimized weight vector
    - b: bias term
    """
    n_samples = K_train.shape[0]
    
    # Initialize weights
    w = np.zeros(n_samples)
    b = 0
    
    # Gradient Descent
    for _ in range(epochs):
        linear_model = K_train @ w + b
        y_pred = sigmoid(linear_model)
        
        # Compute gradients
        error = y_pred - (y_train + 1) / 2  # Convert {-1,1} -> {0,1} for logistic loss
        dw = (K_train.T @ error) / n_samples + C * w
        db = np.mean(error)
        
        # Update weights
        w -= lr * dw
        b -= lr * db

    return w, b


def predict_logistic_regression(K_test, w, b, threshold = 0.5):
    """
    Predict using logistic regression with kernel.
    
    Parameters:
    - K_test: (n_samples, n_train_samples) kernel matrix
    - w: trained weight vector
    - b: bias term
    - threshold: threshold for prediction 
    
    Returns:
    - predictions: {-1,1}
    """
    y_pred_prob = sigmoid(K_test @ w + b)
    return np.where(y_pred_prob >= threshold, 1, -1)


def cross_val_score_manual_logistic_regression(X, y, k, C, threshold, n_splits=3):
    """Performs cross-validation without using sklearn."""
    folds = manual_kfold_split(X, y, n_splits)
    accuracies = []

    for i in range(n_splits):
        val_indices = folds[i]  
        train_indices = np.hstack([folds[j] for j in range(n_splits) if j != i])

        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]

        # Compute kernel matrices
        K_train = spectrum_kernel_matrix(X_train, X_train, k)
        K_val = spectrum_kernel_matrix(X_val, X_train, k)

        # Train and predict
        w, b = train_logistic_regression(K_train, y_train, C)
        predictions = predict_logistic_regression(K_val, w, b, threshold = threshold)

        # Compute accuracy
        accuracy = np.mean(predictions == y_val)
        accuracies.append(accuracy)

    return np.mean(accuracies)


def objective(trial):
    """Objective function for Optuna to optimize k and C."""
    k = trial.suggest_int("k", 2, 6)  # k-mer length between 2 and 6
    C = trial.suggest_float("C", 1e-2, 1e0, log = True)  # Regularization term
    threshold = trial.suggest_float("threshold", 0.48, 0.51)

    cross_val = cross_val_score_manual_logistic_regression(X_train, Y_train, k, C, threshold)

    print(f"C = {C}, k = {k}: {cross_val}")
    return cross_val


def train_and_predict_spectrum_logistic_regression(X_train_path, Y_train_path, X_test_path, n_trials=30):
    """Hyperparameter optimization with Optuna, then train and predict."""
    global X_train, Y_train  
    X_train, Y_train = load_data(X_train_path, Y_train_path)

    df_pred = pd.read_csv(X_test_path)
    X_test = df_pred["seq"].values

    # Run Optuna optimization
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    # Best hyperparameters
    best_k = study.best_params["k"]
    best_C = study.best_params["C"]


    #A MODIFIER !!!
    best_threshold = study.best_params["threshold"]
    print(f"Best k: {best_k}, Best C: {best_C}, Best threshold: {best_threshold}")

    # Train final model
    K_train = spectrum_kernel_matrix(X_train, X_train, best_k)
    K_test = spectrum_kernel_matrix(X_test, X_train, best_k)
    w, b = train_logistic_regression(K_train, Y_train, best_C)

    # Predict on test set
    predictions = predict_logistic_regression(K_test, w, b, threshold = best_threshold)

    # Convert {-1,1} predictions to {0,1}
    predictions = (predictions + 1) // 2

    df_pred["Bound"] = predictions

    return df_pred


# === Run the function with hyperparameter tuning ===
prediction_0_LR = train_and_predict_spectrum_logistic_regression("./data/Xtr0.csv", "./data/Ytr0.csv", "./data/Xte0.csv", n_trials=30)
prediction_1_LR = train_and_predict_spectrum_logistic_regression("./data/Xtr1.csv", "./data/Ytr1.csv", "./data/Xte1.csv", n_trials=30)
prediction_2_LR = train_and_predict_spectrum_logistic_regression("./data/Xtr2.csv", "./data/Ytr2.csv", "./data/Xte2.csv", n_trials=30)

[I 2025-02-24 16:06:21,728] A new study created in memory with name: no-name-b0f00558-8592-4025-8971-68258e4fa337
C:\Users\liuce\AppData\Local\Temp\ipykernel_15328\894395037.py:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
[I 2025-02-24 16:06:24,630] Trial 0 finished with value: 0.49450800125462796 and parameters: {'k': 3, 'C': 0.1522700303229262, 'threshold': 0.5071690092608194}. Best is trial 0 with value: 0.49450800125462796.


C = 0.1522700303229262, k = 3: 0.49450800125462796


[I 2025-02-24 16:06:27,603] Trial 1 finished with value: 0.5160002581291936 and parameters: {'k': 6, 'C': 0.3915350126854863, 'threshold': 0.4886432415948424}. Best is trial 1 with value: 0.5160002581291936.


C = 0.3915350126854863, k = 6: 0.5160002581291936


[I 2025-02-24 16:06:29,976] Trial 2 finished with value: 0.488500494497496 and parameters: {'k': 2, 'C': 0.02769345731545997, 'threshold': 0.5070951983618185}. Best is trial 1 with value: 0.5160002581291936.


...

C = 0.46339290367314545, k = 6: 0.5885120502811657


[I 2025-02-24 16:11:27,773] Trial 28 finished with value: 0.5705135420277849 and parameters: {'k': 4, 'C': 0.01067371317662793, 'threshold': 0.4887295834630342}. Best is trial 22 with value: 0.5970138054096076.


C = 0.01067371317662793, k = 4: 0.5705135420277849


[I 2025-02-24 16:11:31,309] Trial 29 finished with value: 0.583009546277912 and parameters: {'k': 6, 'C': 0.10569969542094641, 'threshold': 0.48607478578110125}. Best is trial 22 with value: 0.5970138054096076.


C = 0.10569969542094641, k = 6: 0.583009546277912
Best k: 6, Best C: 0.04702410502959134, Best threshold: 0.48855778512832665


Best k: 6, Best C: 0.010897299565192597, Best threshold: 0.4891718137318136
'k': 6, 'C': 0.01789216686540087, 'threshold': 0.49165528214733156, 0.57

In [10]:
save_to_csv(prediction_0_LR, prediction_1_LR, prediction_2_LR, suffixe = "lr_separately")

# Prediction only (without Optuna)